## Notes for Aaron: 
==========================================================================================
* You will need to write in the name of your own database in the next cell.


* If you decide to write your own code **ONLY TOUCH THE NCBI STUFF**. I have updated a number of things for JGI, but I haven't combed through it yet, and I know there are some things wrong, so don't use anything other than NCBI.


* One other thing, if you decide to write your own update code, I only have DBNAME, the taxids, and FILEPATH in my database, which is why the code in the following cell just updates taxids instead of replacing all the rows.


* Don't worry about mucking up my database, I have two other copies.


* There are less than 277 rows for NCBI (which is the number of rows you have). There were 17 rows I removed because the strain information was nonexistant (even though they appeared to be labled as strain in the name). These are in a separate table "OTHER". I didn't change anything about the entries, so there should be no need to look at them. But they are there in case you're interested.


* The original code I used to update taxids is after the next cell. You should have all the text files you need in the current directory to get the code to work. However, the database I started with (one of yours) is not here, but the one I made from it is.


* If you find you need to run the cells, you should run them in order, from top to bottom. Variables from previous cells are used in later cells.

# Aaron. Use this code to update your database from mine

It will only update taxids, and the code you want is **ONLY IN THE FIRST CELL**

In [ ]:
# BOILER PLATE -----------------------------------------------------------------------------------------------------------

import sqlite3
import pandas as pd

aaron = sqlite3.connect() #YOUR INPUT IS REQUIRED HERE. WHAT DATABASE WOULD YOU LIKE TO UPDATE

cloeffler = sqlite3.connect("caitlin_database_4-12-2019.db")
ncbi_only = pd.read_sql_query("SELECT * FROM DB_TABLE WHERE DBNAME = 'NCBI'", cloeffler)

# FUNCTIONS --------------------------------------------------------------------------------------------------------------

# MAIN CODE --------------------------------------------------------------------------------------------------------------

for index, row in ncbi_only.iterrows():
    aaron.execute("UPDATE SPECIESDB SET STRAINTAXID = ?, SPECIESTAXID = ?, GENUSTAXID = ? WHERE FILEPATH = ?;", (row['STRAINTAXID'], row['SPECIESTAXID'], row['GENUSTAXID'], row['FILEPATH']))
    aaron.commit()

aaron.close()
cloeffler.close()

# CODE USED TO CORRECT ALL NCBI TAXIDS

#### Make the dictionary with accesion numbers as keys and taxids as values

* **Accesion.txt** is the list of accesion values from the current NCBI references that we have
* **db_info_even_less.txt** is the accesion values paired with taxids. Some of the taxids have decimal values which will be delt with in later cells. Too big for github. Note: this file is too big for jupyter notebook to make a full dictionary out of, so this code only creates dictionary accesion-taxid entries when accesion is also found in **Accesion.txt**.

* If there is more than one taxid to an accesion number, it will state that in the output and print the accesion number

**CREATES dictionary_of_database WHICH IS USED IN LATER CELLS**

In [ ]:
# BOILER PLATE ----------------------------------------------------------------------------------------------------------------



# FUNCTIONS -------------------------------------------------------------------------------------------------------------------

# put all accession numbers into a list
def make_list(filename):
    f = open(filename)
    accession = []
    for line in f: 
        l = line.strip()
        accession.append(l)
    f.close()
    return accession

# Takes in a csv formatted style text file and creates a dictonary from key and value1 and value2
def make_dict(filename, place_key, place_value, only_these):
    f = open(filename)
    db_dict = {}
    for line in f: 
        l = line.strip().split(" ")
        if not l[place_key] in only_these:
            continue
        elif l[place_key] in db_dict.keys():
            if not l[place_value] in db_dict[l[place_key]]:
                db_dict[l[place_key]].append(l[place_value])
                print("This accession number has more than one taxid: " + l[place_key])
            else: 
                continue
        else:
            db_dict[l[place_key]] = [l[place_value]]
    f.close()
    return db_dict

# MAIN FUNCTION ---------------------------------------------------------------------------------------------------------------

these = make_list("Accession.txt") # only want entries for these entries

# format is: Accesion          TaxID
nathan_db = "db_info_even_less.txt"

# keys = accesion, value0 = taxid
dictionary_of_database = make_dict(nathan_db, 0, 1, these)

# print(dictionary_of_database)
print("done making the database")

#### Sort between what needs to be handled, and what can stay the same

* **first_lines.txt** is the first line of every NCBI reference that we have with the filename included. This has the accesion number, organism name, and filename (which is used to create the FILEPATH string that allows us to select the row that must be updated)

**CREATES manual_deal**

A list of entries that will be used to find strain taxid in the next cell.
List is of format: 

\[FILEPATH, suggested taxid, first line of file (contains unedited organism name), suggested taxids from aarons database\]

In [ ]:
# BOILER PLATE -----------------------------------------------------------------------------------------------------------

import math
import sqlite3
import pandas as pd
from ete3 import NCBITaxa
ncbi_taxa = NCBITaxa()

db = sqlite3.connect("my_database_4-12-2019.db")

# FUNCTIONS --------------------------------------------------------------------------------------------------------------
# returns a list in the form: 
# [acccesion, rest of line (with organism name), filename]
def make_list_from_file(filename): 
    f = open(filename)
    list_of_lists = []
    for line in f: 
        l = line.strip().split(":")
        fasta_file_name = l[0]
        first_line = l[1]
        parse = l[1].split(" ")
        A_num = parse[0].strip('>') #Assention number, might have to put '>' in []
        list_of_lists.append([A_num, first_line, fasta_file_name])
    f.close()
    return list_of_lists
    
def make_sql_command(filepath): 
    x = ("select * from DB_TABLE where FILEPATH = \'" + filepath + "\';")
    return x
    
# makes a single string of STRAINTAXID \t SPECIESTAXID \t GENUSTAXID from the sql table
def make_line_from_sql_table(filepath, table):
    x = make_sql_command(filepath)
    t = pd.read_sql_query(x, table)
    r = ''
    for index, row in t.iterrows(): 
        r = (str(row['STRAINTAXID']) + "\t" + str(row['SPECIESTAXID']) + "\t" + str(math.floor(row['GENUSTAXID'])))
    return r # GENUSTAXID gets floored because for some reason it is a float and does not need to be

# Use dictionary_of_database, made in the previous cell, to match accesion numbers to taxid
# returns a taxid as a string or 0 if acc is not a key in the dictionary 
def match_acccesion_to_id(acc, dictionary):
    try:
        taxid = dictionary[acc][0]
        return str(taxid)
    except KeyError: # if the accesion does not exist, returns 0
        return '0'

# MAIN FUNCTIONS ---------------------------------------------------------------------------------------------------------

manual_deal = [] # to hold all the entries and important information that will be used in the next cell

# create a list of lists in the form:
# [acccesion, rest of line, filename.gz]
first_lines = make_list_from_file("first_lines.txt")

for i in first_lines: 
    path = ("/u/home/a/akarlsbe/scratch/fungi/NCBI/" + i[2][0:-3] + "\n") # creates the filepath we use to select rows from sql
    taxid = match_acccesion_to_id(i[0], dictionary_of_database )
    if not taxid.find(".") == -1 or taxid == "0": # strain with no id or dictionary has no match 
        manual_deal.append([path, taxid, i[1], make_line_from_sql_table(path, db)])
    else: # if an integer, could be strain or species
        rank = ncbi_taxa.get_rank([int(taxid)])[int(taxid)]
        if rank == "species": # we already have species IDs, so move on if that is all that is given
            continue
        else: # if not species, want to handle in next cell
            manual_deal.append([path, str(taxid), i[1], make_line_from_sql_table(path, db)])

# Print things ---------------
print("Number of entries that potentially need to be changed: " + str(len(manual_deal)) + "\n\n")

# for i in manual_deal: 
#     print(i)

# Close things ---------------    

db.close()

#### Handles the cases picked from the cell above
This requires some interaction with the user. As it is, the user must make a decision on more than 17 cases, but the code handles all the others. 

When prompted for strain ID there are a few options: 
* "exit" which will exit code and temporarily save current progress (see line 75 and 76 for more detail) 
* "other" inserts the current entry into a list of the same name, these will later be removed from the table and saved into a separate table, also of the same name (OTHER).
* writing in a number will set the strain taxid in the table to that value
* In cases where "Press enter to accept suggested value" appears, if you press "enter" with no information in the box, the suggested taxid will be inserted into the strain ID
* In the case where "press enter..." is **not** seen, pressing enter with nothing in the box will act as if you typed "exit"

**CREATES other**

This is a list of entries where the strain ID information could not be determined. The entries are entries from manual_deal, but the suggested taxids have floats corrected. Other then that, entries are the same. It is used in the next cell. 

In [ ]:
# BOILER PLATE -----------------------------------------------------------------------------------------------------------

import math
import sqlite3
import pandas as pd
from ete3 import NCBITaxa
ncbi_taxa = NCBITaxa()

db = sqlite3.connect("my_database_4-12-2019.db")

# FUNCTIONS --------------------------------------------------------------------------------------------------------------

# get the lineage by taxid and by rank of taxids, return as two separate lists
def get_both_lineages(taxid):         
    try: 
        lineage = ncbi_taxa.get_lineage(taxid) # get taxid lineage
    except ValueError:
        print("ValueError occurred: taxID could not get passed lineage: " + str(taxid))
        return [], []
    except sqlite3.OperationalError:
        print("OperationalError occurred: taxID could not get passed lineage: " + str(taxid))
        return [], []
    else:
        # create list of ranks for the lineage 
        lin_rank = []
        for i in lineage: 
            lin_rank.append(ncbi_taxa.get_rank([i])[i]) # get all the ranks of the  lineage
        return lineage, lin_rank

# Sets the strain Taxid where FILEPATH = filepath in database(product of sqlite3.connect function) 
    
def set_strain(taxid, database, filepath): 
    database.execute("UPDATE DB_TABLE SET STRAINTAXID = ? WHERE FILEPATH = ?", (taxid, filepath))
    database.commit()
    return

# The following functions are used to automate parts of the handling process

# returns True if species is 2 or 3 entries from the end (ie, current taxid is strain)
# "no rank" required to return True bc in ncbi strain is given rank "no rank" (and we are looking for strain)
def auto_accept(lin_rank):
    if lin_rank[-1] == "no rank":
        if lin_rank[-2] == "species" or lin_rank[-3] == "species":
            return True
        else: 
            return False
    else: 
        return False

# given a line and the white space you want to split by, returns string contents at element position
def get_part(line, space, element):
    l = line.split(space)
    return l[element]

# checks if the suggested float taxid matches the species taxid we already have 
def auto_strain_check(all_line): # checks if float is actually strain
    int_id = math.floor(float(all_line[1]))
    current_spec_id = int(get_part(all_line[3], "\t", 1))
    if not int_id == current_spec_id:
        return True
    else: 
        return False

# meant to be run if the function above returns False
# converts suggested taxid from float to integer
def correct_floats(all_line):
    if auto_strain_check(all_line): # if the float is the strain, get rid of decimal
        all_line[1] = str(math.floor(float(all_line[1])))
        return all_line
    else:
        return all_line
    
# MAIN FUNCTIONS ---------------------------------------------------------------------------------------------------------

# other = [] # save feature, comment out when re-running
# done = [] # save feature, comment out when re-running

for i in manual_deal: 
    i = correct_floats(i)
    if not i in done: 
        if i[1] == '0': 
            other.append(i)
            done.append(i)
        elif not (i[1]).find(".") == -1:
#             print(i[0]) #prints the filepath
            print("Suggested taxid: ")
            print(i[1])
            print("Line containing organism name: ")
            print(i[2])
            print("From unedited table: Strain ID\tSpecies ID\tGenus ID")
            print(i[3])
            x = input("strain ID: ")
            if x == "exit" or not x: # exit to exit, and if I press enter with nothing in cell, will exit
                break
            elif x == "other":
                other.append(i)
                done.append(i)
            else: 
                set_strain(int(x), db, i[0])
                print("\n\n")
                done.append(i)
        else: 
            l_i, l_r = get_both_lineages(int(i[1]))
            if auto_accept(l_r): # if not complicated just use given taxid
                set_strain(int(i[1]), db, i[0])
                done.append(i)
            else: 
                print("Suggested taxid: ")
                print(i[1])
                print("Line containing organism name: ")
                print(i[2])
                print("From unedited table: Strain ID\tSpecies ID\tGenus ID")
                print(i[3])
                print("Ranks of suggested taxid lineage: ")
                print(l_r)
                print("Taxids of suggested taxid lineage: ")
                print(l_i)
                print("press ENTER to accept suggested taxid value as new strain ID")
                x = input("strain ID: ") or str(i[1]) # takes input or suggested taxid if box is empty
                if x == "exit":
                    break
                elif x == "other": 
                    other.append(i)
                    done.append(i)
                else: 
                    set_strain(int(x), db, i[0])
                    print("\n\n")
                    done.append(i)


# Print things ---------------

print("This is the number of entries that you placed into \'other\': ")
print(len(other))

print("Yay! You have finished " + str(len(done)) + " entries out of 240!")

# for i in other: 
#     print(i)

# Close things ---------------    

db.close()

#### Removes the entries found in list "other" (created in cell above) will be removed from TABLE_DB and placed into table OTHER

In [ ]:
import sqlite3
import pandas as pd

db = sqlite3.connect("my_database_4-12-2019.db")

for i in other:
    db.execute("INSERT INTO OTHER SELECT * FROM DB_TABLE WHERE FILEPATH = \'" + i[0] + "\';")
    db.commit()
    
    db.execute("DELETE FROM DB_TABLE WHERE FILEPATH = \'" + i[0] + "\';")
    db.commit()
    
# show_me = pd.read_sql_query("SELECT * FROM OTHER;", db)
# print(show_me)

db.close()

# NOT PART OF THE PROGRESSION OF CELLS TO UPDATE NCBI 

#### REFRESH THE NCBI PART OF THE TABLE if you commited errors and want to start over

requires the BASE table where you are pulling taxids. It must have columns for Database name, Filepath, and strain, species, and genus taxids.

Corrected database should already exist

There is a warning, must type yEs (with the capital E) when prompted in order to run the program. (This is just saftey in case cells are run in mindless succession). Anything other than "yEs" will throw assertion error.

In [ ]:
# BOILER PLATE ----------------------------------------------------------------------------------------------------------------

import sqlite3
import pandas as pd

#The warning process ----------------------------------------------------------------------------------------------------------

print("THIS CODE IS TO BE RUN ONLY WHEN YOU WANT TO REFRESH THE NCBI PART OF THE TABLE!")
print("NCBI ROWS WILL BE REPLACED BY BASE DATABASE ROWS FOR NCBI!")
print("THIS WILL UNDO ANYTHING YOU HAVE DONE REGARDING NCBI!")
x = input("WOULD LIKE TO CONTINUE? yEs/no :")

assert(x == 'yEs')


# CONNECT DATABASES -----------------------------------------------------------------------------------------------------------

#connect to Aarons database (BASE)
db = sqlite3.connect('refSeqFungiStatsWithFUNGIDB_NEW_SCHEMA_AND_TAXID_UPDATES-Copy1.db')

data_query = pd.read_sql_query("SELECT * FROM SPECIESDB WHERE DBNAME = 'NCBI';", db)

#connect to own database (The database to be corrected)
mine = sqlite3.connect('my_database_4-12-2019.db')

# FIX TABLE I WILL USE ---------------------------------------------------------------------------------------------------

# remove all rows for NCBI
mine.execute("DELETE FROM DB_TABLE WHERE DBNAME = 'NCBI';")
mine.commit()

# replace deleted rows with BASE data for NCBI 
for index, row in data_query.iterrows():
    mine.execute("INSERT INTO DB_TABLE (DBNAME, SPECIESTAXID, GENUSTAXID, STRAINTAXID, FILEPATH) VALUES (?, ?, ?, ?, ?);", ('NCBI', row['SPECIESTAXID'], row['GENUSTAXID'], 0, row['FILEPATH']))
    mine.commit()

    
# fix the one genus------------------------------------------------------------------------------------------------------
# this is for a specific entry that has no genus in the BASE database I used
spec_old = 373870
spec_new = 2482752
genus = 47747

mine.execute("UPDATE DB_TABLE SET SPECIESTAXID = ?, GENUSTAXID = ? WHERE SPECIESTAXID = ?", (spec_new, genus, spec_old))
mine.commit()

# PRINT NCBI ROWS ----------------------------------------------------------------------------------------------------------------------------

show_me = pd.read_sql_query("SELECT * FROM DB_TABLE WHERE DBNAME='NCBI'", mine)
print(show_me)

# CLOSE FILES -----------------------------------------------------------------------------------------------------------------
mine.close()
db.close()